In [ ]:
def gzgjson_to_pmtiles_dual_layer(geojson_gz, output_pmtiles):
    """Convert a gzipped GeoJSON (FeatureCollection) to PMTiles using tippecanoe.

    This tippecanoe build expects input files (it does NOT accept stdin as "-"),
    so we unpack the .geojson.gz into a temporary .geojson and pass that path.
    """
    import gzip
    import shutil
    import subprocess
    import tempfile
    from pathlib import Path

    geojson_gz = Path(geojson_gz)
    output_pmtiles = Path(output_pmtiles)
    output_pmtiles.parent.mkdir(parents=True, exist_ok=True)
    if not geojson_gz.exists():
        raise FileNotFoundError(f"Input file not found: {geojson_gz}")

    tmp_path = None
    try:
        with gzip.open(geojson_gz, mode="rb") as src, tempfile.NamedTemporaryFile(
            mode="wb", suffix=".geojson", delete=False
        ) as tmp:
            shutil.copyfileobj(src, tmp)
            tmp_path = Path(tmp.name)


        # config von tordans, adjusted by viszim
        cmd = [
            "tippecanoe",
            "-o", str(output_pmtiles),
            "--force",
            '--layer=default',
            # CONFIG:
            # Lowest zoom level for which tiles are generated (default `0`) (`6` is all of Germany on a Laptop, `8` is a litte smaller than a State in Germany)
            '--minimum-zoom=7',
            # https:#github.com/felt/tippecanoe#zoom-levels
            # The automatic --maximum-zoom didn't have the required precision.
            # Instead, we force a maximum-zoom of at least 15 (https:#github.com/felt/tippecanoe#zoom-levels) … or higher.
            #'--smallest-maximum-zoom-guess=15', # <--- HIER verwenden wir 15
            # vizsim: I guess 12 is fine for points
            '--maximum-zoom=12',
            # Increase precision for overzooming https:#github.com/felt/tippecanoe#tile-resolution
            # Note that `--full-detail` does not work for high facotrs like 20 and did not increase the precision enough for lower max zoom
            #'--extra-detail=20', # the value is a factor (not a zoom-value) vizsim: we don't need this for points
            # Don't simplify lines and polygons at maxzoom (but do simplify at lower zooms)
            '--simplify-only-low-zooms',
            # Combine the area of very small polygons into small squares that represent their combined area only at zoom levels below the maximum.
            '--no-tiny-polygon-reduction-at-maximum-zoom',
            # Preserve typology when possible https:#github.com/felt/tippecanoe#line-and-polygon-simplification
            '--no-simplification-of-shared-nodes',
            # https:#github.com/felt/tippecanoe#zoom-levels
            # Increases precision but causes tile drops. => We cannot use this for everything
            # '--generate-variable-depth-tile-pyramid',
            # If you use -rg, it will guess a drop rate that will keep at most 50,000 features in the densest tile https:#github.com/felt/tippecanoe#dropping-a-fixed-fraction-of-features-by-zoom-level
            #'-rg',
            # https:#github.com/felt/tippecanoe#dropping-a-fraction-of-features-to-keep-under-tile-size-limits
            #'--drop-densest-as-needed',
            # vizsim: no drops at all
            "--drop-rate=0",
            # https://github.com/felt/tippecanoe#zoom-levels
            #'--extend-zooms-if-still-dropping',
            # vizsim: enable parallel reading of input
            "--read-parallel",
            str(tmp_path),
        ]
        subprocess.run(cmd, check=True)
    finally:
        if tmp_path is not None:
            tmp_path.unlink(missing_ok=True)

    print(f"✅ PMTiles created: {output_pmtiles}")

In [2]:
geojson_gz = "mk_output/mapillary_markings_bicycle_latest.geojson.gz"
output_pmtiles = "mk_output/mapillary_markings_bicycle_latest.pmtiles"

gzgjson_to_pmtiles_dual_layer(
    geojson_gz=geojson_gz,
    output_pmtiles=output_pmtiles
)

26492 features, 1017616 bytes of geometry and attributes, 496432 bytes of string pool, 0 bytes of vertices, 0 bytes of nodes


✅ PMTiles created: mk_output/mapillary_markings_bicycle_latest.pmtiles


  99.9%  12/2172/1389  
